# BTC trend reversal strategy using Bolinger Bands

- This is a Gaussian Process (GP) optimiser for Bollinger Band strategy
- Instead of a grid search which does an exhaustive search over all parameter space,
  GP only does a search of a limited parameter space and uses heurestics 
  to guess where the global maximums could 
- Optimisation is specified as the number of iterations (tries to get the closer to the best value)
    - Higher the iterations more attemps are made, longer it takes
    - Lower the iterations means you might miss global optimum and stick in a local optimum
- We are using [scikit-optimise](https://scikit-optimize.github.io) Python library as the backend for the optimisation
    - This is way the search space must be defined using [scikit-optimiser Dimension classes: Integer, Real, Categorial](https://scikit-optimize.github.io/stable/modules/generated/skopt.BayesSearchCV.html?highlight=integer%20real)
    - Be wary when using [Real](https://scikit-optimize.github.io/stable/modules/generated/skopt.BayesSearchCV.html?highlight=integer%20real) (float) search space, because indicator results are cached on the disk and searching infinite small steps in floats results to the creation of infinite many indicator data series - it's slow and fills the disk     
    - There is a `perform_optimisation(real_space_rounding)` argument to mitigate the Real unbounded values issue 
- More information about the optimise process below

Further

- Use 1h timeframe
- Backtest on Binance dataset

# Set up

Set up Trading Strategy data client.


In [ ]:
from tradeexecutor.utils.notebook import setup_charting_and_output
from tradingstrategy.client import Client
from tradeexecutor.utils.notebook import setup_charting_and_output, OutputMode

client = Client.create_jupyter_client()

# Set up drawing charts in interactive vector output mode.
# This is slower. See the alternative commented option below.
# setup_charting_and_output(OutputMode.interactive)

# Set up rendering static PNG images.
# This is much faster but disables zoom on any chart.
setup_charting_and_output(OutputMode.static, image_format="png", width=1500, height=1000)


# Parameters

- Strategy parameters define the fixed and [scikit-optimise search space](https://scikit-optimize.github.io/stable/modules/generated/skopt.Space.html?highlight=search%20space)

In [ ]:
import datetime

from skopt.space import Integer, Real, Categorical

from tradingstrategy.chain import ChainId
from tradeexecutor.strategy.default_routing_options import TradeRouting
from tradingstrategy.timebucket import TimeBucket
from tradeexecutor.strategy.cycle import CycleDuration
from tradeexecutor.strategy.parameters import StrategyParameters


class Parameters:
    """Parameteres for this strategy.

    - Collect parameters used for this strategy here

    - Both live trading and backtesting parameters
    """

    id = "binance-btc-bb-optimise" # Used in cache paths

    cycle_duration = CycleDuration.cycle_1h
    candle_time_bucket = TimeBucket.h1
    allocation = 0.98

    adx_length = Integer(10, 70)
    adx_filter_threshold = Integer(0, 5)

    bollinger_bands_ma_length = Integer(10, 40)
    std_dev = Real(0.5, 7)
    
    rsi_bars = Integer(7, 14)  # Number of bars to calculate RSI for each trading bar
    rsi_entry = Integer(30, 70)

    trailing_stop_loss_pct = Real(0.90, 0.99)
    trailing_stop_loss_activation_level = Real(1.02, 1.15)
    stop_loss_pct = Real(0.90, 0.98)
    take_profit_pct = Real(1.05, 1.10)

    #
    # Live trading only
    #
    chain_id = ChainId.polygon
    routing = TradeRouting.default  # Pick default routes for trade execution
    required_history_period = datetime.timedelta(hours=200)

    #
    # Backtesting only
    #
    backtest_start = datetime.datetime(2018, 1, 1)
    backtest_end = datetime.datetime(2024, 5, 1)
    backtest_trading_fee = 0.0005
    stop_loss_time_bucket = TimeBucket.m15
    initial_cash = 10_000


# Trading pairs and market data

- Set up our trading pairs
- Load historical market data for backtesting
- We use Binance CEX data so we have longer history to backtest

In [ ]:
from tradingstrategy.client import Client
from tradingstrategy.lending import LendingProtocolType

from tradeexecutor.utils.binance import create_binance_universe
from tradeexecutor.strategy.trading_strategy_universe import TradingStrategyUniverse, load_partial_data
from tradeexecutor.strategy.execution_context import ExecutionContext, notebook_execution_context
from tradeexecutor.strategy.universe_model import UniverseOptions

# List of trading pairs we use in the backtest
trading_pairs = [
    (ChainId.centralised_exchange, "binance", "BTC", "USDT"),
]

def create_trading_universe(
    timestamp: datetime.datetime,
    client: Client,
    execution_context: ExecutionContext,
    universe_options: UniverseOptions,
) -> TradingStrategyUniverse:
    """Create the trading universe.

    - For live trading, we load DEX data
    """
    strategy_universe = create_binance_universe(
        [f"{p[2]}{p[3]}" for p in trading_pairs],
        candle_time_bucket=Parameters.candle_time_bucket,
        stop_loss_time_bucket=Parameters.stop_loss_time_bucket,
        start_at=universe_options.start_at,
        end_at=universe_options.end_at,
        trading_fee_override=Parameters.backtest_trading_fee,
    )
    return strategy_universe

strategy_universe = create_trading_universe(
    None,
    client,
    notebook_execution_context,
    UniverseOptions.from_strategy_parameters_class(Parameters, notebook_execution_context)
)


# Indicators

- We use `pandas_ta` Python package to calculate technical indicators
- These indicators are precalculated and cached on the disk

In [ ]:
import pandas as pd
import pandas_ta

from tradeexecutor.analysis.regime import Regime
from tradeexecutor.strategy.execution_context import ExecutionContext
from tradeexecutor.strategy.pandas_trader.indicator import IndicatorSet, IndicatorSource, IndicatorDependencyResolver
from tradeexecutor.strategy.parameters import StrategyParameters
from tradeexecutor.strategy.trading_strategy_universe import TradingStrategyUniverse
from tradingstrategy.utils.groupeduniverse import resample_candles

def daily_price(open, high, low, close) -> pd.DataFrame:
    """Resample pricees to daily for ADX filtering."""
    original_df = pd.DataFrame({
        "open": open,
        "high": high,
        "low": low,
        "close": close,
    })    
    daily_df = resample_candles(original_df, pd.Timedelta(days=1))
    return daily_df


def daily_adx(open, high, low, close, length):
    daily_df = daily_price(open, high, low, close)
    adx_df = pandas_ta.adx(
        close=daily_df.close,
        high=daily_df.high,
        low=daily_df.low,
        length=length,
    )
    return adx_df

def regime(
    open, 
    high, low, close, length, regime_threshold,
    dependency_resolver: IndicatorDependencyResolver,
) -> pd.Series:
    """A regime filter based on ADX indicator.

    Get the trend of BTC applying ADX on a daily frame.
    
    - -1 is bear
    - 0 is sideways
    - +1 is bull
    """
    def regime_filter(row):
        # ADX, DMP, # DMN
        average_direction_index, directional_momentum_positive, directional_momentum_negative = row.values

        # We use a threshold to eliminate the noise in ADX,
        # but the threshold can be also zero
        if average_direction_index > regime_threshold:
            # In this case the filter is that if ADX positive is higher than ADX negative,
            # we bullish
            if directional_momentum_positive > directional_momentum_negative:
                return Regime.bull.value
            else:
                return Regime.bear.value
        else:
            return Regime.crab.value

    adx_df = dependency_resolver.get_indicator_data(
        "adx",
        parameters={"length": length},
        column="all",
    )
    # adx_df = daily_adx(open, high, low, close, length)
    regime_signal = adx_df.apply(regime_filter, axis="columns")
    regime_signal = regime_signal.asfreq(Parameters.cycle_duration.to_pandas_timedelta(), method="ffill")
    return regime_signal



def create_indicators(
    timestamp: datetime.datetime | None,
    parameters: StrategyParameters,
    strategy_universe: TradingStrategyUniverse,
    execution_context: ExecutionContext
):
    indicators = IndicatorSet()

    indicators.add(
        "rsi",
        pandas_ta.rsi,
        {"length": parameters.rsi_bars}
    )

    indicators.add(
        "bbands",
        pandas_ta.bbands,
        {"length": parameters.bollinger_bands_ma_length, "std": parameters.std_dev},
    )

    # ADX https://www.investopedia.com/articles/trading/07/adx-trend-indicator.asp
    # https://github.com/twopirllc/pandas-ta/blob/main/pandas_ta/trend/adx.py
    indicators.add(
        "adx",
        pandas_ta.adx,
        {"length": parameters.adx_length},
        IndicatorSource.ohlcv,
        order=1,
    )

    indicators.add(
        "ema",
        pandas_ta.ema,
        {"length": 50},
        IndicatorSource.ohlcv,
    )

    # Price OHLC resampled to daily
    indicators.add(
        "daily_price",
        daily_price,
        {},
        IndicatorSource.ohlcv,
    )

    # A regime filter to detect the trading pair bear/bull markets
    indicators.add(
        "regime",
        regime,
        {"length": parameters.adx_length, "regime_threshold": parameters.adx_filter_threshold},
        IndicatorSource.ohlcv,
        order=2,
    )
        
    return indicators


# Trading algorithm

- Describe out trading strategy as code

In [ ]:
from tradeexecutor.state.visualisation import PlotKind
from tradeexecutor.state.trade import TradeExecution
from tradeexecutor.strategy.pandas_trader.strategy_input import StrategyInput, StrategyInputIndicators


def decide_trades(
    input: StrategyInput,
) -> list[TradeExecution]:

    # 
    # Decidion cycle setup.
    # Read all variables we are going to use for the decisions.
    #
    parameters = input.parameters
    position_manager = input.get_position_manager()
    state = input.state
    timestamp = input.timestamp
    indicators: StrategyInputIndicators = input.indicators
    strategy_universe = input.strategy_universe

    pair = strategy_universe.get_single_pair()
    cash = position_manager.get_current_cash()

    #
    # Indicators
    #

    close_price = indicators.get_price()  # Price the previous 15m candle closed for this decision cycle timestamp
    previous_close_price = indicators.get_price(index=-2) 
    bb_columns = indicators.get_indicator_dataframe("bbands").columns
    # Resolve column names for BB indicator in pandas_ta
    # (It has its own weird naming convention)
    bb_upper_column = next(c for c in bb_columns if c.startswith("BBU"))
    bb_mid_column = next(c for c in bb_columns if c.startswith("BBM"))
    bb_lower_column = next(c for c in bb_columns if c.startswith("BBL"))

    bb_upper = indicators.get_indicator_value("bbands", column=bb_upper_column)
    bb_mid = indicators.get_indicator_value("bbands", column=bb_mid_column)
    bb_lower = indicators.get_indicator_value("bbands", column=bb_lower_column)
    rsi = indicators.get_indicator_value("rsi")
    ema = indicators.get_indicator_value("ema")
    regime_val = indicators.get_indicator_value("regime")
 
    if None in (bb_upper, rsi, close_price, ema):
        # Not enough historic data,
        # cannot make decisions yet
        return []
    
    # If regime filter does not have enough data at the start of the backtest, default to crab market
    if regime_val is None:
        regime_val = 0
        regime = Regime.crab
    else:
        regime = Regime(regime_val)  # Convert to enum for readability
        
    #
    # Trading logic
    #

    trades = []

    # Check for open condition - is the price breaking out
    #
    bb_ratio = (bb_upper - bb_lower) / bb_mid
    if not position_manager.is_any_open():
        if all([
            # close_price > ema,
            regime == Regime.bull,
            previous_close_price < bb_lower,
            close_price > bb_lower,
            rsi > parameters.rsi_entry,
            # bb_ratio > 0.01,
        ]):
            trades += position_manager.open_spot(
                pair,
                value=cash * parameters.allocation,
                stop_loss_pct=parameters.stop_loss_pct,             
            )
    else:        
        # Enable trailing stop loss after we reach the profit taking level
        #
        for position in state.portfolio.open_positions.values():
            if position.trailing_stop_loss_pct is None:
                close_price = indicators.get_price(position.pair)
                if close_price >= position.get_opening_price() * parameters.trailing_stop_loss_activation_level:
                    position.trailing_stop_loss_pct = parameters.trailing_stop_loss_pct                

    # Visualisations
    #
    if input.is_visualisation_enabled():
        visualisation = state.visualisation
        visualisation.plot_indicator(timestamp, "BB upper", PlotKind.technical_indicator_on_price, bb_upper, colour="darkblue")
        visualisation.plot_indicator(timestamp, "BB lower", PlotKind.technical_indicator_on_price, bb_lower, colour="darkblue")
        visualisation.plot_indicator(timestamp, "BB mid", PlotKind.technical_indicator_on_price, bb_mid, colour="blue")
        visualisation.plot_indicator(timestamp, "EMA200", PlotKind.technical_indicator_on_price, ema, colour="orange")

        # Draw the RSI indicator on a separate chart pane.
        # Visualise the high RSI threshold we must exceed to take a position.
        visualisation.plot_indicator(timestamp, "RSI", PlotKind.technical_indicator_detached, rsi)
        visualisation.plot_indicator(timestamp, "Regime", PlotKind.technical_indicator_detached, regime_val)

    return trades  # Return the list of trades we made in this cycle

# Optimise

- Run the optimiser
- Optimise for max CAGR (do not care about the risk)
    - Alternative you could optimise for Sharpe, or any custom 
- Optimiser will automatically use most available CPUs on your computer
- Optimiser has a timeout to detect hung/broken backtests that take forever to complete, see `perform_optimisation(timeout)` argument
    - The default timeout for a single backtest is 10 minutes

In [ ]:
import logging
from tradeexecutor.backtest.optimiser import perform_optimisation
from tradeexecutor.backtest.optimiser import prepare_optimiser_parameters
from tradeexecutor.backtest.optimiser import optimise_profit, optimise_sharpe

# How many Gaussian Process iterations we do
iterations = 40

optimised_results = perform_optimisation(
    iterations=iterations,
    search_func=optimise_profit,
    decide_trades=decide_trades,
    strategy_universe=strategy_universe,
    parameters=prepare_optimiser_parameters(Parameters),  # Handle scikit-optimise search space
    create_indicators=create_indicators,
    # Uncomment for diagnostics
    # log_level=logging.INFO,
    # max_workers=3,
)

print(f"Optimise completed, optimiser searched {optimised_results.get_combination_count()} combinations")

## Results

- Show the top results of all optimiser iterations in a single table

In [ ]:
from tradeexecutor.analysis.grid_search import analyse_grid_search_result
from tradeexecutor.analysis.grid_search import render_grid_search_result_table

#: How many results we analyse from top 
cut_off = 100

all_combinations = optimised_results.get_results_as_grid_search_results() 

df = analyse_grid_search_result(all_combinations[0:cut_off])
print(f"Showing the best {len(df)} results")

render_grid_search_result_table(df)


Render the candidate with the best equity curve.

In [ ]:
from tradeexecutor.visual.grid_search import visualise_single_grid_search_result_benchmark

fig = visualise_single_grid_search_result_benchmark(
    all_combinations[0], 
    strategy_universe, 
    initial_cash=Parameters.initial_cash
)
fig.show()